In [221]:
import pandas as pd
import numpy as np

# Default configuration for time periods in traffic data
input_file = "outputs/2025-12-05/phase_1_current_v16/model_run.csv"
run_folder = 'test_get_toll/phase_1_current_v16'

# Here we load th value of the counts and we multiply the peak hour values by a constant
lights_w = 1

heavies_w = 3
heavies_w_toll = 3
heavies_w_vot = 3

medium_A_w = 3 # TBD: Maybe try 2.5 or 2.75 for every pce value
medium_A_w_toll = 3
medium_A_w_vot = 3

medium_B_w = 3
medium_B_w_toll = 3 # 4
medium_B_w_vot = 3

heavy_A_w = 3
heavy_A_w_toll = 5
heavy_A_w_vot = 3

heavy_B_w = 3
heavy_B_w_toll = 3
heavy_B_w_vot = 3

traffic_condition = 1500
speed_condition = 55

# Default time periods list (for reference)
default_time_periods = [
    "Night",
    "AM-Early",
    "AM-Peak",
    "AM-Shoulder",
    "MD",
    "PM-Shoulder",
    "PM-Peak",
    "PM-Late"
]

# Create the base scenario: hour -> time period mapping
hour_to_period = {
    0: "Night",
    1: "Night",
    2: "Night",
    3: "Night",
    4: "Night",
    5: "AM-Early",
    6: "AM-Peak",
    7: "AM-Shoulder",
    8: "AM-Shoulder",
    9: "AM-Shoulder",
    10: "MD",
    11: "MD",
    12: "MD",
    13: "MD",
    14: "PM-Shoulder",
    15: "PM-Shoulder",
    16: "PM-Peak",
    17: "PM-Peak",
    18: "PM-Late",
    19: "PM-Late",
    20: "PM-Late",
    21: "PM-Late",
    22: "PM-Late",
    23: "Night"
}

# Define the segments and their parameters

awt_adt = 1.1 # Average weekday traffic (AWT) to average daily traffic (ADT) ratio
peak_factor = 1.05 # Peak factor for adjustment at peak hour traffic

hov_percentage = pd.DataFrame({
    'Year' : [2032,2040,2050],
    'HOV percentage' : [0,0,0]
})

hov_percentage.set_index('Year', inplace=True)

# Define segment parameters base
seg_params = pd.DataFrame({
    'SegDir':   ["1NB","1SB","2NB","2SB","3NB","3SB","4NB","4SB","5NB","5SB","6NB","6SB","7NB","7SB","8NB","8SB","9NB","9SB","10NB","10SB"],
    'Length':    [0.7,0.7,0.7,0.7,0.5,0.5,1.6,1.6,2,2,3.6,3.6,2.9,2.9,3.8,3.8,3.4,3.4,4.5,4.5],
    'Inscope':   [0.94,0.94,1,1,1,1,1,1,0.94,0.94,0.94,0.94,0.94,0.94,0.94,0.94,0.94,0.94,0.94,0.94], # [0.82,0.82,0.92,0.92,0.88,0.88,0.88,0.88,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8],  
    'Lanes_GP':  [4]*20, #
    'Lanes_ML':  [2]*20, # Lanes_ML': [2,2,2,2,2,2,2,2,3,3,2,2,2,2], # Do test changing segment 5
    'CapPerLane_GP': [2000]*20,
    'CapPerLane_ML': [1800]*20,
    'Speed_GP':  [55]*6 + [65]*2 + [70]*12,
    'Speed_ML':  [72]*20,
    'Alpha_GP':  [2]*20,
    'Beta_GP':   [8]*20,
    'Alpha_ML':  [2]*20,
    'Beta_ML':   [8]*20,
    'Min_Toll_2016': [None]*20,
    'Max_Toll_2016': [None]*20,
    'LanesGP_AM_Peak': [5]*20,
    'LanesGP_PM_Peak': [5]*20,
})

seg_params.set_index('SegDir', inplace=True)

# Compute capacities as lanes * cap per lane
seg_params['Cap_GP'] = seg_params['Lanes_GP'] * seg_params['CapPerLane_GP']
seg_params['Cap_ML'] = seg_params['Lanes_ML'] * seg_params['CapPerLane_ML']

# Compute peak capacities as Alpha * base capacity
seg_params['CapGP_Peak'] = seg_params['Alpha_GP'] * seg_params['Cap_GP']
seg_params['CapML_Peak'] = seg_params['Alpha_ML'] * seg_params['Cap_ML']

# Optional: if you want integer capacities
seg_params[['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']] = seg_params[
    ['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']
].astype(int)

# Preview
seg_params

,Length,Inscope,Lanes_GP,Lanes_ML,CapPerLane_GP,CapPerLane_ML,Speed_GP,Speed_ML,Alpha_GP,Beta_GP,Alpha_ML,Beta_ML,Min_Toll_2016,Max_Toll_2016,LanesGP_AM_Peak,LanesGP_PM_Peak,Cap_GP,Cap_ML,CapGP_Peak,CapML_Peak
SegDir,,,,,,,,,,,,,,,,,,,,
1NB,0.7,0.94,4,2,2000,1800,55,72,2,8,2,8,None,None,5,5,8000,3600,16000,7200
1SB,0.7,0.94,4,2,2000,1800,55,72,2,8,2,8,None,None,5,5,8000,3600,16000,7200
2NB,0.7,1.00,4,2,2000,1800,55,72,2,8,2,8,None,None,5,5,8000,3600,16000,7200
2SB,0.7,1.00,4,2,2000,1800,55,72,2,8,2,8,None,None,5,5,8000,3600,16000,7200
3NB,0.5,1.00,4,2,2000,1800,55,72,2,8,2,8,None,None,5,5,8000,3600,16000,7200
3SB,0.5,1.00,4,2,2000,1800,55,72,2,8,2,8,None,None,5,5,8000,3600,16000,7200
4NB,1.6,1.00,4,2,2000,1800,65,72,2,8,2,8,None,None,5,5,8000,3600,16000,7200
4SB,1.6,1.00,4,2,2000,1800,65,72,2,8,2,8,None,None,5,5,8000,3600,16000,7200
5NB,2.0,0.94,4,2,2000,1800,70,72,2,8,2,8,None,None,5,5,8000,3600,16000,7200


In [222]:
lookup_period_file = r"inputs/LookUp_Period.csv"

lookup_period = pd.read_csv(
    lookup_period_file,
    sep=",",          # `delimiter` y `sep` son equivalentes; elige uno
    encoding="utf-8",
    decimal=".",      # parsea decimales con punto
    thousands=",",    # parsea separador de miles con coma
    quotechar='"',
    index_col=0
)

# Clip y reasignar
lookup_period = lookup_period * 0

lookup_period


,Bonus/Mile,4 Periods
Period,,
Night,0.0,
AM-Early,0.0,
AM-Peak,0.0,
AM-Shoulder,0.0,
MD,0.0,
PM-Shoulder,0.0,
PM-Peak,0.0,
PM-Late,0.0,


In [223]:
def get_vot(row):

    max_VC = 1.2  # TBD: check if we need to change this value
    ETC_discount = 0.15
    
    captureRateLights =  row["CaptureRateLights"]
    captureRateMediumA =  row["CaptureRateMediumA"]
    captureRateMediumB =  row["CaptureRateMediumB"]
    captureRateHeavyA =  row["CaptureRateHeavyA"]

    tollLights = row["TollLights"]
    tollMediumA = row["TollMediumA"]
    tollMediumB = row["TollMediumB"]
    tollHeavyA = row["TollHeavyA"]

    # ml_pce = seg_params.loc[row["SegDir"], 'Inscope'] * (
    #     row["TotalLights"] * lights_w * captureRateLights + 
    #     row["TotalMediumA"] * medium_A_w * captureRateMediumA +
    #     row["TotalMediumB"] * medium_B_w * captureRateMediumB +
    #     row["TotalHeavyA"] * heavy_A_w * captureRateHeavyA
    # )

    ml_pce = (
        row["InScopeLights"] * lights_w * captureRateLights + 
        row["InScopeMediumA"] * medium_A_w * captureRateMediumA +
        row["InScopeMediumB"] * medium_B_w * captureRateMediumB +
        row["InScopeHeavyA"] * heavy_A_w * captureRateHeavyA
    )
    
    gp_pce = np.min(np.array([row["Corridor PCE"] - ml_pce, row["Suppressed ratio ref"] * seg_params['Cap_GP'][row['SegDir']]]))
    
    speedML = row["Speed ML"] / (1 + row["Alpha ML"] * (((ml_pce + row["HOV3"]) / row["Capacity ML"])** row["Beta ML"])) # TBD: maintain beta ML for future tests

    timeML = 60 * row["Length"] / speedML

    speedGP =  row["Speed GP"] / (1 + row["Alpha GP"] * ((gp_pce / row["Capacity GP"]) ** row["Beta GP"]))

    timeGP = 60 * row["Length"] / speedGP

    timeSavings = timeML - timeGP

    gp_vc = gp_pce / row["Capacity GP"]

    ml_vc = (ml_pce + row["HOV3"]) / row["Capacity ML"]

    # bonusRel = 0.7 * 60 * row["Length"] * ((1/speedGP)-(1/row["Speed GP"])) 

    # bonusAux = (2.5 if (row["Period"] == "AM-Peak" or row["Period"] == "PM-Peak") else 1.6)

    bonusRel = 0.7 * 60 * row["Length"] * ((1/speedGP)-(1/row["Speed GP"]))

    bonusAux = (2.5 if (row["Period"] == "AM-Peak" or row["Period"] == "PM-Peak") else 1.6) 

    bonusSeg = (lookup_period.loc[row["Period"], "Bonus/Mile"] + (bonusAux if gp_vc > 0.3 else 0) * (gp_vc - ml_vc)) * row["Length"] # TBD: add condition for gp_vc

    votLights = -60 * (1 - ETC_discount) * row["Length"] * tollLights / (timeSavings - bonusSeg - bonusRel) # TBD: add household income effect

    votMediumA = -60 * (1 - ETC_discount) * row["Length"] * tollMediumA / (timeSavings - bonusSeg - bonusRel)

    votMediumB = -60 * (1 - ETC_discount) * row["Length"] * tollMediumB / (timeSavings - bonusSeg - bonusRel)

    votHeavyA = -60 * (1 - ETC_discount) * row["Length"] * tollHeavyA / (timeSavings - bonusSeg - bonusRel)

    return pd.Series([votLights, votMediumA, votMediumB, votHeavyA, speedGP, speedML, bonusSeg, timeML, timeGP, timeSavings, bonusRel], index=["VOT Lights", "VOT MediumA", "VOT MediumB", "VOT HeavyA", "Speed GP", "Speed ML", "Bounus Seg", "Time ML", "Time GP", "Time Savings", "BonusRel"])

In [224]:
from scipy.stats import lognorm
from scipy.optimize import minimize, least_squares, Bounds
import numpy as np

ETC_discount = 0.15
max_VC = 1.2  # TBD: check if we need to change this value

def objective_integrated(x, row, tollLights):

    captureRateLights, captureRateMediumA, captureRateMediumB, captureRateHeavyA = x

    tollMediumA = tollLights * medium_A_w_toll
    tollMediumB = tollLights * medium_B_w_toll
    tollHeavyA = tollLights * heavy_A_w_toll

    ml_pce = (
        row["InScopeLights"] * lights_w * captureRateLights + 
        row["InScopeMediumA"] * medium_A_w * captureRateMediumA +
        row["InScopeMediumB"] * medium_B_w * captureRateMediumB +
        row["InScopeHeavyA"] * heavy_A_w * captureRateHeavyA
    )

    gp_pce = np.min(np.array([row["Corridor PCE"] - ml_pce, row["Suppressed ratio ref"] * seg_params['Cap_GP'][row['SegDir']]]))
    
    speedML = row["Speed ML"] / (1 + row["Alpha ML"] * (((ml_pce + row["HOV3"]) / row["Capacity ML"])** row["Beta ML"])) # TBD: maintain beta ML for future tests

    timeML = 60 * row["Length"] / speedML

    speedGP =  row["Speed GP"] / (1 + row["Alpha GP"] * ((gp_pce / row["Capacity GP"]) ** row["Beta GP"]))

    timeGP = 60 * row["Length"] / speedGP

    timeSavings = timeML - timeGP

    gp_vc = gp_pce / row["Capacity GP"]

    ml_vc = (ml_pce + row["HOV3"]) / row["Capacity ML"]

    bonusRel = 0.7 * 60 * row["Length"] * ((1/speedGP)-(1/row["Speed GP"]))

    bonusAux = (2.5 if (row["Period"] == "AM-Peak" or row["Period"] == "PM-Peak") else 1.6)

    bonusSeg = (lookup_period.loc[row["Period"], "Bonus/Mile"] + (bonusAux if gp_vc > 0.3 else 0) * (gp_vc - ml_vc)) * row["Length"] # TBD: add condition for gp_vc

    votLights = -60 * (1 - ETC_discount) * row["Length"] * tollLights / (timeSavings - bonusSeg - bonusRel) # TBD: add household income effect

    votMediumA = -60 * (1 - ETC_discount) * row["Length"] * tollMediumA / (timeSavings - bonusSeg - bonusRel)

    votMediumB = -60 * (1 - ETC_discount) * row["Length"] * tollMediumB / (timeSavings - bonusSeg - bonusRel)

    votHeavyA = -60 * (1 - ETC_discount) * row["Length"] * tollHeavyA / (timeSavings - bonusSeg - bonusRel)

    # TBD: Integrate Reliability
    # Here we compute the lognormal cumulative function for the light vehicles

    mu_lights = row["B1"]

    mu_heavies = mu_lights

    std_dev = row["B2"]

    scale_lights = np.exp(mu_lights)

    scale_heavies = np.exp(mu_heavies) * heavies_w_vot # TBD: change heavies betas (ask Borja)

    # We create a lognormal distribution object

    dist_lights = lognorm(s=std_dev, scale=scale_lights)

    dist_heavies = lognorm(s=std_dev, scale=scale_heavies)

    calcCaptureRateLights = 1 - dist_lights.cdf(votLights)

    calcCaptureMediumA = 1 - dist_heavies.cdf(votMediumA)

    calcCaptureMediumB = 1 - dist_heavies.cdf(votMediumB)

    calcCaptureHeavyA = 1 - dist_heavies.cdf(votHeavyA)

    convergenceLights = calcCaptureRateLights - captureRateLights

    convergenceMediumA = calcCaptureMediumA - captureRateMediumA

    convergenceMediumB = calcCaptureMediumB - captureRateMediumB

    convergenceHeavyA = calcCaptureHeavyA - captureRateHeavyA

    return convergenceLights ** 2 + convergenceMediumA ** 2 + convergenceMediumB ** 2 + convergenceHeavyA ** 2

def optimize_capture(row, M, toll_value):

    capture_first_guess = row["CaptureRateLights"]
    max_capture = row["MaxCapture"]

    ub = [
        max_capture,
        max_capture,
        max_capture,
        max_capture
    ]

    lb = [
        row["MinCapture"],
        row["MinCapture"],
        row["MinCapture"],
        row["MinCapture"]
    ]

    bounds = Bounds(lb=lb, ub=ub)

    x0 = [
        capture_first_guess,
        capture_first_guess,
        capture_first_guess,
        capture_first_guess
    ] # Initial guess, TBD

    Max_iter = 10000000

    # Solve
    result = minimize(
        objective_integrated,
        x0,
        method='trust-constr',
        args=(row,toll_value),
        bounds=bounds,
        # constraints=[nlc],
        options={
            "verbose": 0,
            "maxiter": Max_iter,
            "gtol": 1e-6,
            "xtol": 1e-6,
            "barrier_tol": 1e-6,
            "initial_tr_radius": 1.0,
            "initial_constr_penalty": 1.0,
            "sparse_jacobian": True
        }
    )

    x_opt = result.x
    z_opt = result.fun
    x_opt = np.minimum(bounds.ub, np.maximum(bounds.lb, result.x))
    return x_opt

In [225]:
import pandas as pd
import sys

first_model_df = pd.read_csv(f"{input_file}")

def optimize_row(row):

    # We get the value of the toll for the highest value of revenue

    captureLights = row["CaptureRateLights"]

    captureMediumA = row["CaptureRateMediumA"]

    captureMediumB = row["CaptureRateMediumB"]

    captureHeavyA = row["CaptureRateHeavyA"]

    ml_pce = (row["InScopeLights"] * captureLights + row["HOV3"] + 
              row["InScopeMediumA"] *  captureMediumA * medium_A_w + 
              row["InScopeMediumB"] *  captureMediumB * medium_B_w + 
              row["InScopeHeavyA"] *  captureHeavyA * heavy_A_w)
    
    traffic_lane =  ml_pce/ seg_params.loc[row["SegDir"], 'Lanes_ML']
    # pce_speed = seg_params.loc[row["SegDir"], 'Inscope'] * (row["TotalLights"] * lights_w * captureLights + 
    #                                                         row["TotalMediumA"] * medium_A_w * captureMediumA +
    #                                                         row["TotalMediumB"] * medium_B_w * captureMediumB +
    #                                                         row["TotalHeavyA"] * heavy_A_w * captureHeavyA
    #                                                         )

    pce_speed = (
        row["InScopeLights"] * lights_w * captureLights + 
        row["InScopeMediumA"] * medium_A_w * captureMediumA +
        row["InScopeMediumB"] * medium_B_w * captureMediumB +
        row["InScopeHeavyA"] * heavy_A_w * captureHeavyA
    )

    speedML = row["Speed ML"] / (1 + row["Alpha ML"] * (((pce_speed + row["HOV3"]) / row["Capacity ML"])** row["Beta ML"])) # TBD: maintain beta ML for future tests

    toll = row["TollLights"]

    changed = False

    # print(f'Segment: {row["SegDir"]}, Period: {row["Period"]}, toll: {toll}, traffic: {traffic_lane}, speed: {speedML}')

    x_opt = optimize_capture(row, 1, toll)

    while speedML < 45:
        changed = True
        
        captureLights, captureMediumA, captureMediumB, captureHeavyA = x_opt
        ml_pce = (row["InScopeLights"] * captureLights + row["HOV3"] + 
              row["InScopeMediumA"] *  captureMediumA * medium_A_w + 
              row["InScopeMediumB"] *  captureMediumB * medium_B_w + 
              row["InScopeHeavyA"] *  captureHeavyA * heavy_A_w)
        
        pce_speed = (
            row["InScopeLights"] * lights_w * captureLights + 
            row["InScopeMediumA"] * medium_A_w * captureMediumA +
            row["InScopeMediumB"] * medium_B_w * captureMediumB +
            row["InScopeHeavyA"] * heavy_A_w * captureHeavyA
        )

        traffic_lane =  ml_pce/ seg_params.loc[row["SegDir"], 'Lanes_ML']
        speedML = row["Speed ML"] / (1 + row["Alpha ML"] * (((pce_speed + row["HOV3"]) / row["Capacity ML"])** row["Beta ML"])) # TBD: maintain beta ML for future tests
        # print(f'Segment: {row["SegDir"]}, Period: {row["Period"]}, toll: {toll}, traffic: {traffic_lane}, speed: {speedML}')
        objective_integrated(x_opt, row, toll)
        toll += 0.05

        if toll > 14:
            break

    if changed:
        changed = False
        print(f'Segment: {row["SegDir"]}, Period: {row["Period"]}, toll: {toll}, traffic: {traffic_lane}, speed: {speedML}')

    return pd.Series([captureLights, captureMediumA, captureMediumB, captureHeavyA, toll], 
                index=["CaptureRateLights", "CaptureRateMediumA", "CaptureRateMediumB", "CaptureRateHeavyA", "TollLights"])

first_model_df[["CaptureRateLights", "CaptureRateMediumA", "CaptureRateMediumB", "CaptureRateHeavyA", "TollLights"]] = first_model_df.apply(
    optimize_row, axis=1, result_type='expand'
)

C:\Users\crisp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\scipy\optimize\_differentiable_functions.py:376: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


Segment: 2NB, Period: PM-Shoulder, toll: 14.049999907678135, traffic: 1620.894326443845, speed: 38.611197511240086
Segment: 3SB, Period: AM-Shoulder, toll: 14.049999941486005, traffic: 1580.306579460876, speed: 42.20494950139193
Segment: 4NB, Period: AM-Shoulder, toll: 4.187484382763139, traffic: 759.8116385225676, speed: 71.85513772075844
Segment: 5NB, Period: PM-Peak, toll: 3.66192953849251, traffic: 892.7122534052304, speed: 71.47675301086295
Segment: 6NB, Period: AM-Early, toll: 3.575022195734969, traffic: 646.6910817796726, speed: 71.96005003656526
Segment: 6SB, Period: PM-Peak, toll: 14.049999997663521, traffic: 1628.2014046428344, speed: 37.96613205624469
Segment: 8NB, Period: AM-Peak, toll: 14.049999960741234, traffic: 1569.8715003587045, speed: 43.12619817195445


In [226]:
def get_share(row):

    ml_pce = (row["InScopeLights"] * row["CaptureRateLights"] + row["HOV3"] + 
              row["InScopeMediumA"] *  row["CaptureRateMediumA"] * medium_A_w + 
              row["InScopeMediumB"] *  row["CaptureRateMediumB"] * medium_B_w + 
              row["InScopeHeavyA"] *  row["CaptureRateHeavyA"] * heavy_A_w)
    
    tollShare = 0
    
    devShare = 0

    tdotShare = 0

    baseToll = row["TollLights"]

    if row["TollLights"] > 2 * row["MaxToll"]:
        tollShare = row["TollLights"] - row["MaxToll"]
        
        if ml_pce > 1500 and ml_pce < 1650:
            tdotShare = tollShare
        else:
            devShare = 0.1 * tollShare
            tdotShare = 0.9 * tollShare
        
        baseToll = row["MaxToll"]

    elif row["TollLights"] > row["MaxToll"]:
        tollShare = row["TollLights"] - row["MaxToll"]
        
        if ml_pce > 1500 and ml_pce < 1650:
            devShare = 0.25 * tollShare
            tdotShare = 0.75 * tollShare
        else:
            devShare = 0.5 * tollShare
            tdotShare = 0.5 * tollShare

        baseToll = row["MaxToll"]

    return pd.Series([baseToll, devShare, tdotShare], index=["baseToll", "devTollShare", "tdotTollShare"])   
    

In [227]:
first_model_df[["baseToll","devTollShare", "tdotTollShare"]] = first_model_df.apply(
    get_share, axis=1, result_type='expand'
)

first_model_df["TollMediumA"] = first_model_df.apply(
    lambda row: row["TollLights"] * medium_A_w_toll,
    axis=1
)

first_model_df["TollMediumB"] = first_model_df.apply(
    lambda row: row["TollLights"] * medium_B_w_toll,
    axis=1
)

first_model_df["TollHeavyA"] = first_model_df.apply(
    lambda row: row["TollLights"] * heavy_A_w_vot,
    axis=1
)

first_model_df["devTollMediumA"] = first_model_df.apply(
    lambda row: row["devTollShare"] * medium_A_w_toll,
    axis=1
)

first_model_df["devTollMediumB"] = first_model_df.apply(
    lambda row: row["devTollShare"] * medium_B_w_toll,
    axis=1
)

first_model_df["devTollHeavyA"] = first_model_df.apply(
    lambda row: row["devTollShare"] * heavy_A_w_vot,
    axis=1
)

first_model_df["tdotTollMediumA"] = first_model_df.apply(
    lambda row: row["tdotTollShare"] * medium_A_w_toll,
    axis=1
)

first_model_df["tdotTollMediumB"] = first_model_df.apply(
    lambda row: row["tdotTollShare"] * medium_B_w_toll,
    axis=1
)

first_model_df["tdotTollHeavyA"] = first_model_df.apply(
    lambda row: row["tdotTollShare"] * heavy_A_w_vot,
    axis=1
)

first_model_df["baseTollMediumA"] = first_model_df.apply(
    lambda row: row["baseToll"] * medium_A_w_toll,
    axis=1
)

first_model_df["baseTollMediumB"] = first_model_df.apply(
    lambda row: row["baseToll"] * medium_B_w_toll,
    axis=1
)

first_model_df["baseTollHeavyA"] = first_model_df.apply(
    lambda row: row["baseToll"] * heavy_A_w_vot,
    axis=1
)

first_model_df[["VOT Lights", "VOT MediumA", "VOT MediumB", "VOT HeavyA", "Speed GP Real", "Speed ML Real", "Bonus Seg", "Time ML", "Time GP", "Time Savings", "BonusRel"]] = first_model_df.apply(
    get_vot, axis=1, result_type='expand'
)

first_model_df

,Unnamed: 0,Year,SegDir,Segment,Direction,Period,Hours/Day,Peak,4Periods,Length,...,tdotTollShare,devTollMediumA,devTollMediumB,devTollHeavyA,tdotTollMediumA,tdotTollMediumB,tdotTollHeavyA,baseTollMediumA,baseTollMediumB,baseTollHeavyA
0,0,2025,1NB,1,NB,Night,6,OP,NT,0.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.817708,0.817708,0.817708
1,1,2025,1NB,1,NB,AM-Early,1,OP,AM,0.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.700889,2.700889,2.700889
2,2,2025,1NB,1,NB,AM-Peak,1,Peak,AM,0.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.799984,4.799984,4.799984
3,3,2025,1NB,1,NB,AM-Shoulder,3,OP,AM,0.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.799923,4.799923,4.799923
4,4,2025,1NB,1,NB,MD,4,OP,MD,0.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.799994,4.799994,4.799994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,603,2050,8SB,8,SB,AM-Shoulder,3,OP,AM,3.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.725000,7.725000,7.725000
508,604,2050,8SB,8,SB,MD,4,OP,MD,3.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.725000,7.725000,7.725000
509,605,2050,8SB,8,SB,PM-Shoulder,2,OP,PM,3.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.024984,11.024984,11.024984
510,606,2050,8SB,8,SB,PM-Peak,2,Peak,PM,3.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.449993,15.449993,15.449993


In [228]:
first_model_df["Reliability %"] = first_model_df.apply(
    lambda row: row["BonusRel"] / (row["BonusRel"] + row["Bonus Seg"] - row["Time Savings"]),
    axis=1
)

first_model_df["Bonus %"] = first_model_df.apply(
    lambda row: row["Bonus Seg"] / (row["BonusRel"] + row["Bonus Seg"] - row["Time Savings"]),
    axis=1
)

first_model_df["Time Savings %"] = first_model_df.apply(
    lambda row: - row["Time Savings"] / (row["BonusRel"] + row["Bonus Seg"] - row["Time Savings"]),
    axis=1
)

first_model_df["MLVeh_Lights"] = first_model_df.apply(
    lambda row: row["InScopeLights"] * row["CaptureRateLights"],
    axis=1
)

first_model_df["GPVeh_Lights"] = first_model_df.apply(
    lambda row: row["TotalLights"] - row["MLVeh_Lights"] - row["HOV3"],
    axis=1
)

first_model_df["MLVeh_MediumA"] = first_model_df.apply(
    lambda row: row["InScopeMediumA"] * row["CaptureRateMediumA"],
    axis=1
)

first_model_df["GPVeh_MediumA"] = first_model_df.apply(
    lambda row: row["TotalMediumA"] - row["MLVeh_MediumA"],
    axis=1
)

first_model_df["MLVeh_MediumB"] = first_model_df.apply(
    lambda row: row["InScopeMediumB"] * row["CaptureRateMediumB"],
    axis=1
)

first_model_df["GPVeh_MediumB"] = first_model_df.apply(
    lambda row: row["TotalMediumB"] - row["MLVeh_MediumB"],
    axis=1
)

first_model_df["MLVeh_HeavyA"] = first_model_df.apply(
    lambda row: row["InScopeHeavyA"] * row["CaptureRateHeavyA"],
    axis=1
)

first_model_df["GPVeh_HeavyA"] = first_model_df.apply(
    lambda row: row["TotalHeavyA"] - row["MLVeh_HeavyA"],
    axis=1
)

first_model_df["MLVeh"] = first_model_df.apply(
    lambda row: row["MLVeh_Lights"] + row["MLVeh_MediumA"] + row["MLVeh_MediumB"] + row["MLVeh_HeavyA"],
    axis=1
)

first_model_df["GPVeh"] = first_model_df.apply(
    lambda row: row["GPVeh_Lights"] + row["GPVeh_MediumA"] + row["GPVeh_MediumB"] + row["GPVeh_HeavyA"] + row["TotalHeavyB"],
    axis=1
)

first_model_df["GPVehDay"] = first_model_df.apply(
    lambda row: row["GPVeh"] * row["Hours/Day"],
    axis=1
)

first_model_df["ML PCE"] = first_model_df.apply(
    lambda row:row["MLVeh_Lights"] * lights_w
    + row["MLVeh_MediumA"] * medium_A_w
    + row["MLVeh_MediumB"] * medium_B_w
    + row["MLVeh_HeavyA"] * heavy_A_w, # TBD: is HOV included in the PCE?
    axis=1
)

first_model_df["ML V/C"] = first_model_df.apply(
    lambda row:row["ML PCE"] / row["Capacity ML"],
    axis=1
)

first_model_df["ML Volume"] = first_model_df.apply(
    lambda row: row["ML PCE"] * row["Hours/Day"],
    axis=1
)

first_model_df["ML PCE Total"] = first_model_df.apply(
    lambda row: row["ML PCE"] * row["Hours/Day"] * row["Length"],
    axis=1
)

first_model_df["GP PCE"] = first_model_df.apply(
    lambda row:row["GPVeh_Lights"] * lights_w
    + row["GPVeh_MediumA"] * medium_A_w
    + row["GPVeh_MediumB"] * medium_B_w
    + row["GPVeh_HeavyA"] * heavy_A_w
    + row["TotalHeavyB"] * heavy_B_w, # TBD: is HOV included in the PCE?
    axis=1
)

first_model_df["GP Volume"] = first_model_df.apply(
    lambda row: row["GP PCE"] * row["Hours/Day"],
    axis=1
)

first_model_df["GP PCE Total"] = first_model_df.apply(
    lambda row: row["GP PCE"] * row["Hours/Day"] * row["Length"],
    axis=1
)

# Corridor Vals

first_model_df["Highway PCE"] = first_model_df.apply(
    lambda row: row["ML PCE"] + row["GP PCE"],
    axis=1
)

first_model_df["Highway Volume"] = first_model_df.apply(
    lambda row: row["ML Volume"] + row["GP Volume"],
    axis=1
)

first_model_df["Highway PCE Total"] = first_model_df.apply(
    lambda row: row["ML PCE Total"] + row["GP PCE Total"],
    axis=1
)

first_model_df["Highway V/C"] = first_model_df.apply(
    lambda row: (row["ML PCE"] + row["GP PCE"]) / (row["Capacity GP"]),
    axis=1
)

# We add the TollPerSeg column

first_model_df["TollLightsPerSeg"] = first_model_df.apply(
    lambda row: row["TollLights"] * row["Length"],
    axis=1
)

first_model_df["TollMediumAPerSeg"] = first_model_df.apply(
    lambda row: row["TollMediumA"] * row["Length"],
    axis=1
)

first_model_df["TollMediumBPerSeg"] = first_model_df.apply(
    lambda row: row["TollMediumB"] * row["Length"],
    axis=1
)

first_model_df["TollHeavyAPerSeg"] = first_model_df.apply(
    lambda row: row["TollHeavyA"] * row["Length"],
    axis=1
)

# Toll blend
first_model_df["TollBlend"] = first_model_df.apply(
    lambda row: (row["TollLights"] * row["MLVeh_Lights"] 
                + row["TollMediumA"] * row["MLVeh_MediumA"]
                + row["TollMediumB"] * row["MLVeh_MediumB"]
                + row["TollHeavyA"] * row["MLVeh_HeavyA"]
                ) / (row["MLVeh"]),
    axis=1
)

first_model_df["RevenuePerHour"] = first_model_df.apply(
    lambda row: row["TollLightsPerSeg"] * row["MLVeh_Lights"]
    + row["TollMediumAPerSeg"] * row["MLVeh_MediumA"]
    + row["TollMediumBPerSeg"] * row["MLVeh_MediumB"]
    + row["TollHeavyAPerSeg"] * row["MLVeh_HeavyA"],
    axis=1
)

############### Revenue Stream #######################

first_model_df["DevRevenuePerDay"] = first_model_df.apply(
    lambda row: (row["devTollShare"] * row["MLVeh_Lights"]
    + row["devTollMediumA"] * row["MLVeh_MediumA"]
    + row["devTollMediumB"] * row["MLVeh_MediumB"]
    + row["devTollHeavyA"] * row["MLVeh_HeavyA"]) * row["Length"] * row["Hours/Day"],
    axis=1
)

first_model_df["TDOTRevenuePerDay"] = first_model_df.apply(
    lambda row: (row["tdotTollShare"] * row["MLVeh_Lights"]
    + row["tdotTollMediumA"] * row["MLVeh_MediumA"]
    + row["tdotTollMediumB"] * row["MLVeh_MediumB"]
    + row["tdotTollHeavyA"] * row["MLVeh_HeavyA"]) * row["Length"] * row["Hours/Day"],
    axis=1
)

first_model_df["BaseRevenuePerDay"] = first_model_df.apply(
    lambda row: (row["baseToll"] * row["MLVeh_Lights"]
    + row["baseTollMediumA"] * row["MLVeh_MediumA"]
    + row["baseTollMediumB"] * row["MLVeh_MediumB"]
    + row["baseTollHeavyA"] * row["MLVeh_HeavyA"]) * row["Length"] * row["Hours/Day"],
    axis=1
)

######################################################

first_model_df["RevenuePerHourLights"] = first_model_df.apply(
    lambda row: row["TollLightsPerSeg"] * row["MLVeh_Lights"],
    axis=1
)

first_model_df["RevenuePerHourMediumA"] = first_model_df.apply(
    lambda row: row["TollMediumAPerSeg"] * row["MLVeh_MediumA"],
    axis=1
)

first_model_df["RevenuePerHourMediumB"] = first_model_df.apply(
    lambda row: row["TollMediumBPerSeg"] * row["MLVeh_MediumB"],
    axis=1
)

first_model_df["RevenuePerHourHeavyA"] = first_model_df.apply(
    lambda row: row["TollHeavyAPerSeg"] * row["MLVeh_HeavyA"],
    axis=1
)

first_model_df["RevenuePerDay"] = first_model_df.apply(
    lambda row: row["RevenuePerHour"] * row["Hours/Day"],
    axis=1
)

first_model_df["RevenuePerDayLights"] = first_model_df.apply(
    lambda row: row["RevenuePerHourLights"] * row["Hours/Day"],
    axis=1
)

first_model_df["RevenuePerDayMediumA"] = first_model_df.apply(
    lambda row: row["RevenuePerHourMediumA"] * row["Hours/Day"],
    axis=1
)

first_model_df["RevenuePerDayMediumB"] = first_model_df.apply(
    lambda row: row["RevenuePerHourMediumB"] * row["Hours/Day"],
    axis=1
)

first_model_df["RevenuePerDayHeavyA"] = first_model_df.apply(
    lambda row: row["RevenuePerHourHeavyA"] * row["Hours/Day"],
    axis=1
)

first_model_df["TransactionsDay_Lights"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["MLVeh_Lights"],
    axis=1
)

first_model_df["TransactionsDay_MediumA"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["MLVeh_MediumA"],
    axis=1
)

first_model_df["TransactionsDay_MediumB"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["MLVeh_MediumB"],
    axis=1
)

first_model_df["TransactionsDay_HeavyA"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["MLVeh_HeavyA"],
    axis=1
)

first_model_df["TransactionsDay"] = first_model_df.apply(
    lambda row: row["TransactionsDay_Lights"] + row["TransactionsDay_MediumA"] + row["TransactionsDay_MediumB"] + row["TransactionsDay_HeavyA"],
    axis=1
)

first_model_df["GPDay_Lights"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["GPVeh_Lights"],
    axis=1
)

first_model_df["GPDay_MediumA"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["GPVeh_MediumA"],
    axis=1
)

first_model_df["GPDay_MediumB"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["GPVeh_MediumB"],
    axis=1
)

first_model_df["GPDay_HeavyA"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["GPVeh_HeavyA"],
    axis=1
)

first_model_df["GPDay_HeavyB"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["TotalHeavyB"],
    axis=1
)


first_model_df["TransactionsLightsLength"] = first_model_df.apply(
    lambda row: row["TransactionsDay_Lights"] * row["Length"],
    axis=1
)

first_model_df["TransactionsMediumALength"] = first_model_df.apply(
    lambda row: row["TransactionsDay_MediumA"] * row["Length"],
    axis=1
)

first_model_df["TransactionsMediumBLength"] = first_model_df.apply(
    lambda row: row["TransactionsDay_MediumB"] * row["Length"],
    axis=1
)

first_model_df["TransactionsHeavyALength"] = first_model_df.apply(
    lambda row: row["TransactionsDay_HeavyA"] * row["Length"],
    axis=1
)

first_model_df["TotalVeh_hours"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["TotalVeh"],
    axis=1
)

first_model_df["GPDayLength_Lights"] = first_model_df.apply(
    lambda row: row["Length"] * row["GPDay_Lights"],
    axis=1
)

first_model_df["GPDayLength_MediumA"] = first_model_df.apply(
    lambda row: row["Length"] * row["GPDay_MediumA"],
    axis=1
)

first_model_df["GPDayLength_MediumB"] = first_model_df.apply(
    lambda row: row["Length"] * row["GPDay_MediumB"],
    axis=1
)

first_model_df["GPDayLength_HeavyA"] = first_model_df.apply(
    lambda row: row["Length"] * row["GPDay_HeavyA"],
    axis=1
)

first_model_df["GPDayLength_HeavyB"] = first_model_df.apply(
    lambda row: row["Length"] * row["GPDay_HeavyB"],
    axis=1
)

# Extra ones for the pivot

first_model_df["Difference in Travel Time"] = first_model_df.apply(
    lambda row: row["Time Savings"] / row["Time GP"],
    axis=1
)

first_model_df["Implied Min VOT"] = first_model_df.apply(
    lambda row: 60 * row["TollLightsPerSeg"] / (row["Time GP"] - row["Time ML"]),
    axis=1
)

# first_model_df.to_csv(f'I24_study/{today_str}/model_run.csv')

first_model_df.to_csv(f'{run_folder}/model_run.csv')

first_model_df

,Unnamed: 0,Year,SegDir,Segment,Direction,Period,Hours/Day,Peak,4Periods,Length,...,devTollHeavyA,tdotTollMediumA,tdotTollMediumB,tdotTollHeavyA,baseTollMediumA,baseTollMediumB,baseTollHeavyA,DevRevenuePerDay,TDOTRevenuePerDay,BaseRevenuePerDay
0,0,2025,1NB,1,NB,Night,6,OP,NT,0.7,...,0.0,0.0,0.0,0.0,0.817708,0.817708,0.817708,0.0,0.0,232.335395
1,1,2025,1NB,1,NB,AM-Early,1,OP,AM,0.7,...,0.0,0.0,0.0,0.0,2.700889,2.700889,2.700889,0.0,0.0,467.759867
2,2,2025,1NB,1,NB,AM-Peak,1,Peak,AM,0.7,...,0.0,0.0,0.0,0.0,4.799984,4.799984,4.799984,0.0,0.0,2077.550613
3,3,2025,1NB,1,NB,AM-Shoulder,3,OP,AM,0.7,...,0.0,0.0,0.0,0.0,4.799923,4.799923,4.799923,0.0,0.0,3319.828753
4,4,2025,1NB,1,NB,MD,4,OP,MD,0.7,...,0.0,0.0,0.0,0.0,4.799994,4.799994,4.799994,0.0,0.0,3689.345277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,603,2050,8SB,8,SB,AM-Shoulder,3,OP,AM,3.8,...,0.0,0.0,0.0,0.0,7.725000,7.725000,7.725000,0.0,0.0,38590.504690
508,604,2050,8SB,8,SB,MD,4,OP,MD,3.8,...,0.0,0.0,0.0,0.0,7.725000,7.725000,7.725000,0.0,0.0,53728.096353
509,605,2050,8SB,8,SB,PM-Shoulder,2,OP,PM,3.8,...,0.0,0.0,0.0,0.0,11.024984,11.024984,11.024984,0.0,0.0,74542.638532
510,606,2050,8SB,8,SB,PM-Peak,2,Peak,PM,3.8,...,0.0,0.0,0.0,0.0,15.449993,15.449993,15.449993,0.0,0.0,108012.501546


In [229]:
period_order = [
    "Night",
    "AM-Early",
    "AM-Peak",
    "AM-Shoulder",
    "MD",
    "PM-Shoulder",
    "PM-Peak",
    "PM-Late"
]

first_model_df["Period"] = pd.Categorical(
    first_model_df["Period"],
    categories=period_order,
    ordered=True
)

first_model_df["GP V/C normal"] = first_model_df.apply(
    lambda row: row["GP PCE"] / seg_params.loc[row["SegDir"], 'Cap_GP'],
    axis=1
)

first_model_df["GP V/C capacity factors"] = first_model_df.apply(
    lambda row: row["GP PCE"] / row["Capacity GP"],
    axis=1
)

first_model_df["CaptureRateLights"] = first_model_df.apply(
    lambda row: row["CaptureRateLights"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

C:\Users\crisp\AppData\Local\Temp\ipykernel_21628\29839758.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  first_model_df["GP V/C normal"] = first_model_df.apply(
C:\Users\crisp\AppData\Local\Temp\ipykernel_21628\29839758.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  first_model_df["GP V/C capacity factors"] = first_model_df.apply(


In [230]:
# Define a light green style
def apply_light_green_style(df):
    return (df.style
    .set_table_styles([{
        'selector': 'th',
        'props': [
            ('background-color', "#7afc7f"),  # Light green
            ('color', '000000'),
            ('font-weight', 'bold'),
            ('border', '1px solid black')
        ]
    }])
    .set_properties(**{'border': '1px solid black'})  # Optional: add borders to data cells
)

In [231]:
# model_file = 'outputs/2025-10-13/base_gdt_v2/model_run.csv'

# first_model_df = pd.read_csv(model_file)

# First Column

def generate_pivot(df, pivot_file, years):

    with pd.ExcelWriter(pivot_file, engine='openpyxl') as writer:

        for year_val in years:

            df_filtered = df[df["Year"] == year_val]

            lengths = df_filtered.pivot_table(
                index="Period",  # Single row index
                columns=["Direction", "SegDir"],
                values="Length",
                aggfunc='first'  # Takes the first value (since they're all the same)
            ).round(2)

            lengths = df_filtered[['Segment', 'Length']].drop_duplicates().set_index("Segment")

            lengths = lengths.T

            sumTollLights = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TollLights"
            ).round(2)

            sumCaptureRateLights = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="CaptureRateLights"
            ).round(2)

            revenuePerPeriod = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="RevenuePerDay"
            ).round(2)

            revenuePerPeriod_LV = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="RevenuePerDayLights"
            ).round(2)

            revenuePerPeriod_MediumA = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="RevenuePerDayMediumA"
            ).round(2)

            revenuePerPeriod_MediumB = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="RevenuePerDayMediumB"
            ).round(2)

            revenuePerPeriod_HeavyA = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="RevenuePerDayHeavyA"
            ).round(2)

            percentage_revenue = revenuePerPeriod / revenuePerPeriod.sum()

            # Column 2

            ML_PCE_miles = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="ML PCE Total"
            ).round(2)

            ML_Volume = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="ML Volume"
            ).round(2)

            ML_flow = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="ML PCE"
            ).round(2)

            ML_vc = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="ML V/C"
            ).round(2)

            speed_ML = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Speed ML Real"
            ).round(2)

            ML_light_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsDay_Lights"
            ).round(2)

            ML_MediumA_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsDay_MediumA"
            ).round(2)

            ML_MediumB_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsDay_MediumB"
            ).round(2)

            ML_HeavyA_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsDay_HeavyA"
            ).round(2)

            ML_light_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsLightsLength"
            ).round(2)

            ML_MediumA_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsMediumALength"
            ).round(2)

            ML_MediumB_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsMediumBLength"
            ).round(2)

            ML_HeavyA_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsHeavyALength"
            ).round(2)

            time_ML = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Time ML"
            ).round(2)

            time_savings_vot = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Time Savings %"
            ).round(2)

            reliability = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Reliability %"
            ).round(2)

            bonus_vot = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Bonus %"
            ).round(2)

            # Column 3

            GP_PCE_miles = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GP PCE Total"
            ).round(2)

            GP_Volume = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GP Volume"
            ).round(2)

            GP_flow = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GP PCE"
            ).round(2)

            sum_GP_VC = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GP V/C normal"
            ).round(2)

            sum_GP_VC_nominal = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GP V/C capacity factors"
            ).round(2)

            speed_GP = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Speed GP Real"
            ).round(2)

            time_GP = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Time GP"
            ).round(2)

            GP_light_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GPDay_Lights"
            ).round(2)

            GP_MediumA_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GPDay_MediumA"
            ).round(2)

            GP_MediumB_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsDay_MediumB"
            ).round(2)

            GP_HeavyA_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsDay_HeavyA"
            ).round(2)

            GP_HeavyB_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsDay_HeavyA"
            ).round(2)

            GP_light_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsLightsLength"
            ).round(2)

            GP_MediumA_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsMediumALength"
            ).round(2)

            GP_MediumB_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsMediumBLength"
            ).round(2)

            GP_HeavyA_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsHeavyALength"
            ).round(2)

            GP_HeavyB_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsHeavyALength"
            ).round(2)

            # Column 4

            corridor_PCE_total = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Highway PCE Total"
            ).round(2)

            corridor_volume = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Highway Volume"
            ).round(2)

            corridor_flow = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Highway PCE"
            ).round(2)

            corridor_vc = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Highway V/C"
            ).round(2)

            time_savings = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Time Savings"
            ).round(2)

            difference_time = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Difference in Travel Time"
            ).round(2)

            implied_vot = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Implied Min VOT"
            ).round(2)

            toll_lights_seg = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TollLightsPerSeg"
            ).round(2)

            toll_mediumA_seg = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TollMediumAPerSeg"
            ).round(2)

            # sum_GPVeh = df_filtered.pivot(
            #     index="Period",
            #     columns=["Direction", "Segment"],
            #     values="GPVeh"
            # )

            # sum_MLVeh = df_filtered.pivot(
            #     index="Period",
            #     columns=["Direction", "Segment"],
            #     values="MLVeh"
            # )

            # sumRevHour = df_filtered.pivot(
            #     index="Period",
            #     columns=["Direction", "Segment"],
            #     values="RevenuePerHour"
            # )

            pivot_vals_column_1 = [sumTollLights, sumCaptureRateLights, revenuePerPeriod, revenuePerPeriod_LV, revenuePerPeriod_MediumA, revenuePerPeriod_MediumB, revenuePerPeriod_HeavyA, percentage_revenue]

            keys_1 = ["TollLights", "CaptureRateLights", "Revenue Per Period", "Revenue Per Period Lights", "Revenue Per Period Medium A", "Revenue Per Period Medium B", "Revenue Per Period Heavy A", "% Workday Revenue"]
            
            pivot_vals_1 = dict(zip(keys_1, pivot_vals_column_1))

            pivot_vals_column_2 = [ML_PCE_miles, ML_Volume, ML_flow, ML_vc, speed_ML, time_ML, ML_light_veh_mile, ML_light_veh_period, ML_MediumA_veh_mile, ML_MediumA_veh_period, ML_MediumB_veh_mile, ML_MediumB_veh_period, ML_HeavyA_veh_mile, ML_HeavyA_veh_period, time_savings_vot, reliability, bonus_vot]
            
            keys_2 = ["ML PCE.Miles", "ML Volume (PCE/Period)", "ML Flow (PCE/hr)", "ML V/C", "ML Speed", "ML time", "ML light veh.Miles", "ML light veh/period", "ML medium A veh.Miles", "ML medium A veh/period", "ML medium B veh.Miles", "ML medium B veh/period", "ML heavy A veh.Miles", "ML heavy A veh/period", "Time Savings %", "Reliability %", "Bonus %"]

            pivot_vals_2 = dict(zip(keys_2, pivot_vals_column_2))
            
            pivot_vals_column_3 = [GP_PCE_miles, GP_Volume, GP_flow, sum_GP_VC, speed_GP, time_GP, GP_light_veh_mile, GP_light_veh_period, GP_MediumA_veh_mile, GP_MediumA_veh_period, GP_MediumB_veh_mile, GP_MediumB_veh_period, GP_HeavyA_veh_mile, GP_HeavyA_veh_period, GP_HeavyB_veh_mile, GP_HeavyB_veh_period]

            keys_3 = ["GP PCE.Miles", "GP Volume (PCE/Period)", "GP Flow (PCE/hr)", "GP V/C real", "GP Speed", "GP Time", "GP light veh.Miles", "GP light veh/period", "GP medium A veh.Miles", "GP medium A veh/period", "GP medium B veh.Miles", "GP medium B veh/period", "GP heavy A veh.Miles", "GP heavy A veh/period", "GP heavy B veh.Miles", "GP heavy B veh/period"]
            
            pivot_vals_3 = dict(zip(keys_3, pivot_vals_column_3))

            pivot_vals_column_4 = [corridor_PCE_total, corridor_volume, corridor_flow, corridor_vc, time_savings, difference_time, toll_lights_seg, toll_mediumA_seg, implied_vot]

            keys_4 = ["Highway PCE.Miles", "Highway Volume", "Highway Flow", "Highway Vol / GP Capacity", "Time Saving", "Difference in travel time", "LVr Toll", "Medium A Toll", "Implied VOT"]

            pivot_vals_4 = dict(zip(keys_4, pivot_vals_column_4))

            pd.DataFrame().to_excel(writer, sheet_name=f'Year_{year_val}')

            startcol = 1

            start_row = 0

            worksheet = writer.sheets[f'Year_{year_val}']

            worksheet.cell(start_row + 1, startcol + 1, "Lenght")

            styled_table = lengths

            # styled_table = styled_table.format("{:.2f}")

            styled_table.to_excel(writer, sheet_name=f'Year_{year_val}', startrow=start_row + 1, startcol=startcol)

            start_row += lengths.shape[0] + 3

            for key, pivot_vals in pivot_vals_1.items():

                worksheet = writer.sheets[f'Year_{year_val}']

                worksheet.cell(start_row + 1, startcol + 1, key)

                start_row += 1

                start_column_cumm = pivot_vals.shape[1] - 1

                styled_table = apply_light_green_style(pivot_vals)

                styled_table = styled_table.format("{:.2f}")

                styled_table.to_excel(writer, sheet_name=f'Year_{year_val}', startrow=start_row, startcol=startcol)

                start_row += pivot_vals.shape[0] + 5
            
            startcol += start_column_cumm + 3

            start_row = 4
        
            for key, pivot_vals in pivot_vals_2.items():

                worksheet = writer.sheets[f'Year_{year_val}']

                worksheet.cell(start_row + 1, startcol + 1, key)

                start_row += 1

                start_column_cumm = pivot_vals.shape[1]

                styled_table = apply_light_green_style(pivot_vals)

                styled_table = styled_table.format("{:.2f}")

                styled_table.to_excel(writer, sheet_name=f'Year_{year_val}', startrow=start_row, startcol=startcol)

                start_row += pivot_vals.shape[0] + 5
            
            startcol += start_column_cumm + 3

            start_row = 4

            for key, pivot_vals in pivot_vals_3.items():

                worksheet = writer.sheets[f'Year_{year_val}']

                worksheet.cell(start_row + 1, startcol + 1, key)

                start_row += 1

                start_column_cumm = pivot_vals.shape[1]

                styled_table = apply_light_green_style(pivot_vals)

                styled_table = styled_table.format("{:.2f}")

                styled_table.to_excel(writer, sheet_name=f'Year_{year_val}', startrow=start_row, startcol=startcol)

                start_row += pivot_vals.shape[0] + 5
            
            startcol += start_column_cumm + 3

            start_row = 4

            for key, pivot_vals in pivot_vals_4.items():

                worksheet = writer.sheets[f'Year_{year_val}']

                worksheet.cell(start_row + 1, startcol + 1, key)

                start_row += 1

                start_column_cumm = pivot_vals.shape[1]

                styled_table = apply_light_green_style(pivot_vals)

                styled_table = styled_table.format("{:.2f}")

                styled_table.to_excel(writer, sheet_name=f'Year_{year_val}', startrow=start_row, startcol=startcol)

                start_row += pivot_vals.shape[0] + 5
            
            startcol += start_column_cumm + 3

In [232]:
years = [2025,2032,2040,2050]

generate_pivot(first_model_df, f'{run_folder}/pivot.xlsx', years)

C:\Users\crisp\AppData\Local\Temp\ipykernel_21628\616045681.py:15: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  lengths = df_filtered.pivot_table(
C:\Users\crisp\AppData\Local\Temp\ipykernel_21628\616045681.py:126: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  ML_light_veh_mile = df_filtered.pivot_table(
C:\Users\crisp\AppData\Local\Temp\ipykernel_21628\616045681.py:132: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  ML_MediumA_veh_mile = df_filtered.pivot_table(
C:\Users\crisp\AppData\Local\Temp\ipyker

In [233]:
def generate_revenue_stream(final_df, file_name):

    output_df = pd.DataFrame(columns=['Year', 'Total Revenue', 'Base Revenues', 'Dev Revenues', 'TDOT Revenues', 'Total Transactions', 'AADT', 'Toll/AADT/mi', 'Capture']) # AADT refers to ML

    years = [2025, 2032, 2040, 2050]

    Anualization_factor = 295 # TBD: Just changed from 290, check if it makes sense
    traffic_anualization = 310 # TBD: Check if it fits
    year_days = 365

    length_correction_factor = 1.055

    for year in years:

        yearly_df = final_df[final_df["Year"] == year]

        # We compute the revenues

        revenues_df = yearly_df.groupby(['Segment', 'Direction'])[["RevenuePerDay"]].sum().reset_index() #

        revenues_dev = yearly_df.groupby(['Segment', 'Direction'])[["DevRevenuePerDay"]].sum().reset_index()

        revenues_tdot = yearly_df.groupby(['Segment', 'Direction'])[["TDOTRevenuePerDay"]].sum().reset_index()

        revenues_base = yearly_df.groupby(['Segment', 'Direction'])[["BaseRevenuePerDay"]].sum().reset_index()

        revenues_df["Annual Revenue"] = revenues_df["RevenuePerDay"] * Anualization_factor # We have to change 290 for an annual constant

        revenues_dev["Annual Revenue"] = revenues_dev["DevRevenuePerDay"] * Anualization_factor

        revenues_tdot["Annual Revenue"] = revenues_tdot["TDOTRevenuePerDay"] * Anualization_factor

        revenues_base["Annual Revenue"] = revenues_base["BaseRevenuePerDay"] * Anualization_factor

        total_revenues = revenues_df["Annual Revenue"].sum() * length_correction_factor # TBD: Change 1.02 for a parameter

        total_revenues_dev = revenues_dev["Annual Revenue"].sum() * length_correction_factor

        total_revenues_tdot = revenues_tdot["Annual Revenue"].sum() * length_correction_factor

        total_revenues_base = revenues_base["Annual Revenue"].sum() * length_correction_factor

        # We compute the total transactions

        transactions_df = yearly_df.groupby(['Segment', 'Direction'])[["TransactionsDay"]].sum().reset_index()

        # transactions_df_aux.append(transactions_df)

        total_transactions = transactions_df["TransactionsDay"].sum() * traffic_anualization # TBD: Change 315 for a parameter

        # We compute the AADT

        aadt_df = yearly_df.groupby(['SegDir'])[["TransactionsDay"]].sum()

        gp_traffic_df = yearly_df.groupby(['SegDir'])[["GPVehDay"]].sum()

        total_df = yearly_df.groupby(['SegDir'])[["TotalVeh_hours"]].sum()

        df_lengths = pd.DataFrame(seg_params["Length"])

        merged_df = pd.merge(aadt_df, df_lengths, on='SegDir')

        merged_gp_traffic_df = pd.merge(gp_traffic_df, df_lengths, on='SegDir')

        # Compute product
        merged_df['Weighted'] = merged_df['Length'].values * merged_df["TransactionsDay"].values

        merged_gp_traffic_df['Weighted'] = merged_gp_traffic_df['Length'] * merged_gp_traffic_df["GPVehDay"]

        # Sum weighted values
        total_aadt = (traffic_anualization/year_days) * merged_df['Weighted'].sum() / (merged_df['Length'].sum()/2)

        traffic_gp = (traffic_anualization/year_days) * merged_gp_traffic_df['Weighted'].sum() / (merged_gp_traffic_df['Length'].sum()/2)

        gp_ml = total_aadt + traffic_gp

        capture_total = total_aadt / gp_ml

        toll_AADT_mi = total_revenues / (year_days * total_aadt * (merged_df['Length'].sum()/2))
        
        output_df.loc[len(output_df)] = [year, total_revenues, total_revenues_base, total_revenues_dev, total_revenues_tdot, total_transactions, total_aadt, toll_AADT_mi, capture_total]

    # Define full range of years to interpolate over
    full_years = pd.DataFrame({"Year": np.arange(int(output_df["Year"].min()), int(output_df["Year"].max()) + 1)})

    # Merge with original to create missing years with NaNs
    merged = pd.merge(full_years, output_df, on="Year", how="left")

    # Interpolate all numeric columns except "Year"
    interpolated = np.log(merged).interpolate(method="linear")

    interpolated = np.exp(interpolated)

    # Copy original column names
    original_columns = list(interpolated.columns)

    # Track how many columns we've inserted to adjust the index
    insert_count = 0

    # Start from index 1 to skip the first column (index 0)
    for i in range(1, len(original_columns)):
        col = original_columns[i]
        new_col_name = f"Var {col} (%)"
        insert_position = i + insert_count + 1  # Adjust position with insert_count
        interpolated[col] = interpolated[col].round(6)
        new_col_val = interpolated[col].pct_change()
        interpolated.insert(insert_position, new_col_name, new_col_val)
        insert_count += 1

    interpolated.to_csv(file_name)

    return interpolated

In [234]:
interpolated = generate_revenue_stream(first_model_df, f'{run_folder}/revenue_stream.csv')

interpolated

C:\Users\crisp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pandas\core\internals\blocks.py:395: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


,Year,Total Revenue,Var Total Revenue (%),Base Revenues,Var Base Revenues (%),Dev Revenues,Var Dev Revenues (%),TDOT Revenues,Var TDOT Revenues (%),Total Transactions,Var Total Transactions (%),AADT,Var AADT (%),Toll/AADT/mi,Var Toll/AADT/mi (%),Capture,Var Capture (%)
0,2025.0,2.038720e+08,NaN,2.038720e+08,NaN,0.000000e+00,NaN,0.000000e+00,NaN,8.179643e+07,NaN,27991.076044,NaN,1.262955,NaN,0.178454,NaN
1,2026.0,2.208087e+08,0.083075,2.208087e+08,0.083075,0.000000e+00,NaN,0.000000e+00,NaN,8.484547e+07,0.037276,28801.846930,0.028965,1.329370,0.052587,0.179312,0.004808
2,2027.0,2.391524e+08,0.083075,2.391524e+08,0.083075,0.000000e+00,NaN,0.000000e+00,NaN,8.800817e+07,0.037276,29636.102066,0.028965,1.399277,0.052587,0.180175,0.004813
3,2028.0,2.590200e+08,0.083075,2.590200e+08,0.083075,0.000000e+00,NaN,0.000000e+00,NaN,9.128877e+07,0.037276,30494.521681,0.028965,1.472860,0.052586,0.181042,0.004812
4,2029.0,2.805382e+08,0.083075,2.805382e+08,0.083075,0.000000e+00,NaN,0.000000e+00,NaN,9.469164e+07,0.037276,31377.805707,0.028965,1.550313,0.052587,0.181913,0.004811
5,2030.0,3.038439e+08,0.083075,3.038439e+08,0.083075,0.000000e+00,NaN,0.000000e+00,NaN,9.822137e+07,0.037276,32286.674351,0.028965,1.631839,0.052587,0.182788,0.004810
6,2031.0,3.290858e+08,0.083075,3.290858e+08,0.083075,0.000000e+00,NaN,0.000000e+00,NaN,1.018827e+08,0.037276,33221.868679,0.028965,1.717652,0.052587,0.183668,0.004814
7,2032.0,3.564247e+08,0.083075,3.564247e+08,0.083075,0.000000e+00,NaN,0.000000e+00,NaN,1.056804e+08,0.037276,34184.151224,0.028965,1.807978,0.052587,0.184551,0.004808
8,2033.0,3.807149e+08,0.068150,3.807149e+08,0.068150,0.000000e+00,NaN,0.000000e+00,NaN,1.080581e+08,0.022498,34951.157632,0.022437,1.888810,0.044709,0.186700,0.011644
9,2034.0,4.066605e+08,0.068150,4.066605e+08,0.068150,0.000000e+00,NaN,0.000000e+00,NaN,1.104892e+08,0.022498,35735.373734,0.022437,1.973257,0.044709,0.188873,0.011639
